좋아요. 애매하게 두루뭉술한 가이드가 아니라, **바로 돌릴 수 있는 최소-노력 실전 스크립트**를 4가지 태스크(분류/요약/대화/RAG)별로 드릴게요.
(전제: Windows+Conda, 캐시는 `D:\WorkSpace\Hands_on_Gen_AI`)

---

### 공통 사전 준비 (한 번만)

```powershell
pip install -U transformers accelerate peft bitsandbytes sentence-transformers datasets faiss-cpu
# NVIDIA GPU가 없거나 bitsandbytes 문제면: pip uninstall -y bitsandbytes  (옵션)
setx HF_HOME "D:\WorkSpace\Hands_on_Gen_AI"
setx TRANSFORMERS_CACHE "D:\WorkSpace\Hands_on_Gen_AI"
```

---

## 1) 문서 분류 (한국어/멀티) — LoRA로 가볍게 미세튜닝

* 모델: `klue/roberta-base` (국문 강함) 또는 `xlm-roberta-base` (멀티)
* 입력: 간단한 CSV(`train.csv`) 가정: `text,label` 컬럼

```python
# train_classification_lora.py
import os, torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model

CACHE = r"D:\WorkSpace\Hands_on_Gen_AI"
MODEL_ID = "klue/roberta-base"  # or "xlm-roberta-base"
NUM_LABELS = 2

tok = AutoTokenizer.from_pretrained(MODEL_ID, cache_dir=CACHE)
base = AutoModelForSequenceClassification.from_pretrained(MODEL_ID, num_labels=NUM_LABELS, cache_dir=CACHE)

peft_cfg = LoraConfig(
    r=8, lora_alpha=16, lora_dropout=0.1, bias="none",
    target_modules=["query","value","key","dense"]  # RoBERTa 계열 공통적용 OK
)
model = get_peft_model(base, peft_cfg)

ds = load_dataset("csv", data_files={"train":"train.csv","valid":"valid.csv"})
def tok_fn(ex): return tok(ex["text"], truncation=True, padding="max_length", max_length=256)
ds = ds.map(tok_fn)
ds = ds.rename_column("label", "labels")
ds.set_format(type="torch", columns=["input_ids","attention_mask","labels"])

args = TrainingArguments(
    output_dir="./out-cls", per_device_train_batch_size=16, per_device_eval_batch_size=32,
    num_train_epochs=3, fp16=torch.cuda.is_available(), evaluation_strategy="epoch",
    save_strategy="epoch", logging_steps=20, load_best_model_at_end=True, metric_for_best_model="eval_loss"
)

trainer = Trainer(model=model, args=args, train_dataset=ds["train"], eval_dataset=ds["valid"])
trainer.train()
trainer.save_model("./out-cls-best")
tok.save_pretrained("./out-cls-best")
```

**추론 스니펫**

```python
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
tok = AutoTokenizer.from_pretrained("./out-cls-best")
mdl = AutoModelForSequenceClassification.from_pretrained("./out-cls-best")
clf = pipeline("text-classification", model=mdl, tokenizer=tok)
print(clf("이 제품 정말 마음에 들어요"))  # [{'label': 'LABEL_1', 'score': ...}]
```

---

## 2) 요약 — FLAN-T5 베이스로 가볍게

* 모델: `google/flan-t5-base` (영/다국어 텍스트 요약에 실전성 좋음)

```python
# summarize_quick.py
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import os
CACHE = r"D:\WorkSpace\Hands_on_Gen_AI"
MODEL_ID = "google/flan-t5-base"

tok = AutoTokenizer.from_pretrained(MODEL_ID, cache_dir=CACHE)
mdl = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID, cache_dir=CACHE)
summarizer = pipeline("summarization", model=mdl, tokenizer=tok)

text = """
여기에 긴 문서 본문을 붙이세요. 여러 문단 가능.
핵심만 깔끔히 뽑고 싶을 때 유용합니다.
"""
print(summarizer(text, max_length=160, min_length=60, do_sample=False, num_beams=4, no_repeat_ngram_size=3))
```

> 한국어 길면 `google/mt5-base`로 교체 가능. 도메인 적합화가 필요하면 LoRA로 문서-요약 페어 몇 천 샘플만으로도 미세튜닝 효과적.

---

## 3) 대화/지시수행 — 소형 인스트럭트 LLM

* 모델 예시: `Qwen/Qwen2-7B-Instruct`, `meta-llama/Meta-Llama-3-8B-Instruct`, `google/gemma-2-9b-it`

```python
# chat_quick.py
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
CACHE = r"D:\WorkSpace\Hands_on_Gen_AI"
MODEL_ID = "Qwen/Qwen2-7B-Instruct"  # 원하시는 걸로 교체

tok = AutoTokenizer.from_pretrained(MODEL_ID, cache_dir=CACHE, use_fast=True)
mdl = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, cache_dir=CACHE, torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)
gen = pipeline("text-generation", model=mdl, tokenizer=tok)

system = "당신은 간결하고 유용하게 답하는 한국어 비서입니다."
user = "RAG를 한 문장으로 설명해줘."
prompt = f"{system}\n\n사용자: {user}\n어시스턴트:"
out = gen(prompt, max_new_tokens=200, temperature=0.7, top_p=0.9, do_sample=True)
print(out[0]["generated_text"])
```

> 저사양이면 GGUF(llama.cpp)나 4/8bit 양자화(AWQ/GPTQ/BNB)로 메모리 절약. 긴 컨텍스트가 필요하면 해당 크기/버전 옵션 확인.

---

## 4) RAG — E5 임베딩 + FAISS + LLM 리더

* **핵심**: 임베딩(검색) 품질이 절반 이상을 좌우. `intfloat/e5-base`로 간단히 시작.

```python
# rag_minimal.py
from sentence_transformers import SentenceTransformer
import faiss, numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

CACHE = r"D:\WorkSpace\Hands_on_Gen_AI"
EMB = "intfloat/e5-base"  # 임베딩
LLM = "Qwen/Qwen2-7B-Instruct"  # 생성기 (교체 가능)

# 1) 문서 인덱싱
docs = [
    "RAG는 검색으로 문맥을 보강한 후 생성하는 기법이다.",
    "임베딩 품질이 RAG 성능에 큰 영향을 준다.",
    "FAISS는 빠른 벡터 검색 라이브러리다.",
]
embed = SentenceTransformer(EMB, cache_folder=CACHE)
doc_vecs = embed.encode([f"passage: {d}" for d in docs], normalize_embeddings=True)
index = faiss.IndexFlatIP(doc_vecs.shape[1]); index.add(doc_vecs)

# 2) 쿼리 → 검색
query = "RAG가 뭐야?"
qv = embed.encode([f"query: {query}"], normalize_embeddings=True)
D, I = index.search(qv, k=3)
ctx = "\n\n".join([docs[i] for i in I[0]])

# 3) LLM으로 답 생성(컨텍스트 주입)
tok = AutoTokenizer.from_pretrained(LLM, cache_dir=CACHE, use_fast=True)
mdl = AutoModelForCausalLM.from_pretrained(LLM, cache_dir=CACHE, torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32, device_map="auto")
gen = pipeline("text-generation", model=mdl, tokenizer=tok)

prompt = f"""다음 컨텍스트를 바탕으로 사용자의 질문에 답하세요.
컨텍스트:
{ctx}

질문: {query}
답변:"""
resp = gen(prompt, max_new_tokens=200, temperature=0.2, top_p=0.9, do_sample=True)
print(resp[0]["generated_text"])
```

> 품질 업그레이드: 문서 chunking(예: 300\~600토큰), top-k 3\~5, MMR 다양성, **Cross-Encoder rerank** 추가, 컨텍스트 길이 확장 가능한 LLM 사용.

---

## 선택 가이드 (짧게)

* **라벨 분류/태깅/스팸·감성** → (1) 분류 LoRA
* **요약/정제/형식 변환** → (2) 요약 T5
* **대화/작성/지시수행** → (3) 인스트럭트 LLM
* **사내문서/웹지식+정확한 답** → (4) RAG

원하시면 위 네 가지 중 **지금 바로 돌릴 태스크 하나**를 골라,

* 데이터 예시(샘플 CSV 생성),
* 학습/평가 커맨드,
* 배포(로컬 API/vLLM/llama.cpp)
  까지 한 번에 맞춰서 더 깔끔하게 드릴게요.


# DO LoRA

좋아요. 제가 \*\*문서 분류(한국어 감성 분류 예시)\*\*를 골라서, 지금 바로 돌릴 수 있게 **데이터 생성 → LoRA 미세튜닝 → 평가 → 추론**까지 한 번에 준비해드릴게요.
(전제: Windows/Conda, 캐시 `D:\WorkSpace\Hands_on_Gen_AI`)

---

## 0) 필수 패키지 + 캐시 경로 (한 번만)

```powershell
pip install -U transformers accelerate peft datasets scikit-learn
setx HF_HOME "D:\WorkSpace\Hands_on_Gen_AI"
setx TRANSFORMERS_CACHE "D:\WorkSpace\Hands_on_Gen_AI"
```

> 터미널/IDE 재시작 후 진행 권장

---

## 1) 샘플 데이터 만들기 (train/valid CSV 자동 생성)

아래 스크립트가 **작은 한국어 감성 데이터셋**을 생성해 `train.csv`, `valid.csv`를 만듭니다.

* 컬럼: `text,label` (0=부정, 1=긍정)

```python
# make_data.py
import csv, os, random

random.seed(42)
train_data_pos = [
    "정말 만족스러워요", "제품이 기대 이상이에요", "친절하고 빠른 서비스네요", "다음에도 또 살게요",
    "품질이 아주 좋아요", "설명이 이해하기 쉬워요", "디자인이 마음에 들어요", "가격 대비 최고입니다"
]
train_data_neg = [
    "별로에요", "품질이 실망스럽네요", "설치가 너무 어려웠어요", "다시는 안 살래요",
    "설명이 불친절해요", "가격이 너무 비싸요", "반품하고 싶어요", "추천하기 어렵네요"
]

def make_split(pos, neg, ratio=0.75):
    size_pos = int(len(pos) * ratio)
    size_neg = int(len(neg) * ratio)
    tr = [(t,1) for t in random.sample(pos, len(pos))[:size_pos]] + [(t,0) for t in random.sample(neg, len(neg))[:size_neg]]
    va = [(t,1) for t in pos[size_pos:]] + [(t,0) for t in neg[size_neg:]]
    random.shuffle(tr); random.shuffle(va)
    return tr, va

train, valid = make_split(train_data_pos, train_data_neg, ratio=0.75)

def write_csv(path, rows):
    with open(path, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["text","label"])
        for t,l in rows:
            w.writerow([t, l])

os.makedirs("./data", exist_ok=True)
write_csv("./data/train.csv", train)
write_csv("./data/valid.csv", valid)
print("Wrote ./data/train.csv and ./data/valid.csv")
```

실행:

```powershell
python make_data.py
```

---

## 2) LoRA 미세튜닝 스크립트 (KLUE RoBERTa)

* 모델: `klue/roberta-base` (한국어 태스크에 강함)
* **LoRA**로 가볍게 튜닝 → VRAM/시간 절약

```python
# train_classification_lora.py
import os, torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer
)
from peft import LoraConfig, get_peft_model
from sklearn.metrics import accuracy_score, f1_score

CACHE = r"D:\WorkSpace\Hands_on_Gen_AI"
MODEL_ID = "klue/roberta-base"
NUM_LABELS = 2
MAX_LEN = 128

tok = AutoTokenizer.from_pretrained(MODEL_ID, cache_dir=CACHE)
base = AutoModelForSequenceClassification.from_pretrained(
    MODEL_ID, num_labels=NUM_LABELS, cache_dir=CACHE
)

# LoRA 설정 (RoBERTa 계열 공통적으로 잘 먹는 모듈들)
peft_cfg = LoraConfig(
    r=8, lora_alpha=16, lora_dropout=0.1, bias="none",
    target_modules=["query","key","value","dense"]
)
model = get_peft_model(base, peft_cfg)

# CSV 로드
ds = load_dataset("csv", data_files={"train":"./data/train.csv","valid":"./data/valid.csv"})

def tok_fn(batch):
    return tok(batch["text"], truncation=True, padding="max_length", max_length=MAX_LEN)

ds = ds.map(tok_fn, batched=True)
ds = ds.rename_column("label", "labels")
ds.set_format(type="torch", columns=["input_ids","attention_mask","labels"])

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="macro"),
    }

args = TrainingArguments(
    output_dir="./out-cls",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    fp16=torch.cuda.is_available(),
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
)

trainer = Trainer(
    model=model, args=args,
    train_dataset=ds["train"], eval_dataset=ds["valid"],
    compute_metrics=compute_metrics
)
trainer.train()
trainer.save_model("./out-cls-best")
tok.save_pretrained("./out-cls-best")
print("Saved to ./out-cls-best")
```

실행:

```powershell
python train_classification_lora.py
```

---

## 3) 평가/추론 (학습 결과 사용)

```python
# infer_classification.py
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

tok = AutoTokenizer.from_pretrained("./out-cls-best")
mdl = AutoModelForSequenceClassification.from_pretrained("./out-cls-best")

clf = pipeline("text-classification", model=mdl, tokenizer=tok, return_all_scores=False)
samples = [
    "정말 만족스러워요",
    "별로에요",
    "가격은 비싸지만 품질은 좋아요",
]
for s in samples:
    print(s, "->", clf(s))
```

실행:

```powershell
python infer_classification.py
```

---

## 4) 바로 적용 팁 (작업에 맞게 바꾸기)

* **라벨 수 변경**: `NUM_LABELS`를 바꾸고 CSV의 `label` 범위를 맞추세요.
* **데이터 늘리기**: `./data/train.csv`, `./data/valid.csv`에 행만 추가하면 됩니다(수천\~수만 행까지).
* **길이 조절**: 긴 문서는 `MAX_LEN`을 256\~512로 늘리되, 배치/VRAM을 상황에 맞게 줄이세요.
* **속도/메모리**: GPU 없으면 `fp16=False`(자동), 배치 축소. 더 줄이고 싶다면 4/8bit 로드시 bitsandbytes를 추가로 세팅할 수 있습니다.
* **멀티 클래스**: `NUM_LABELS` 늘리고 CSV에 해당 라벨을 사용하세요.

---

원하시면 위 분류 파이프라인을 **당신의 실제 데이터 구조**(칼럼명/라벨 체계)로 맞춰 스크립트까지 즉시 변환해 드릴게요. 데이터 예시 몇 줄만 보내주시면 됩니다.


# DO RAG

좋아요! \*\*RAG(검색 증강 생성)\*\*을 바로 돌릴 수 있게, **문서 준비 → 인덱싱 → 질의 → 생성(출처 표기)** 까지 “최소 노력” 세트로 드릴게요.
(전제: Windows/Conda, 캐시는 `D:\WorkSpace\Hands_on_Gen_AI`, 로컬 LLM은 **Qwen2-7B-Instruct** 예시. GPU 없으면 더 작은 모델로 바꾸세요.)

---

## 0) 설치 & 캐시 경로 (한 번만)

```powershell
pip install -U transformers accelerate sentence-transformers faiss-cpu
# 선택(재순위기 추가): pip install -U cross-encoder
setx HF_HOME "D:\WorkSpace\Hands_on_Gen_AI"
setx TRANSFORMERS_CACHE "D:\WorkSpace\Hands_on_Gen_AI"
```

---

## 1) 지식 문서 샘플 만들기

작동 확인용 KB(지식베이스) 문서 몇 개를 만듭니다.

```python
# make_kb.py
import os, textwrap
os.makedirs("./kb", exist_ok=True)

docs = {
    "rag_intro.md": """
RAG(Retrieval-Augmented Generation)은 외부 문서를 검색하여
그 결과를 컨텍스트로 주입한 뒤 답변을 생성하는 기법이다.
최신성·사실성·출처 제시가 필요한 QA에 유용하다.
핵심 구성은 임베딩/벡터DB(FAISS 등), 리트리버, 리더(LLM)이다.
""",
    "embeddings.md": """
임베딩 모델(E5, BGE 등)은 텍스트를 고차원 벡터로 변환한다.
좋은 임베딩일수록 의미적으로 가까운 텍스트들이 벡터 공간에서도 가깝다.
RAG에서 임베딩 품질은 검색/정답률에 큰 영향을 준다.
""",
    "faiss.md": """
FAISS는 Facebook AI가 만든 고성능 벡터 검색 라이브러리이다.
CPU 버전만으로도 수만~수백만 스케일에서 빠른 유사도 검색을 제공한다.
지표로는 내적 또는 코사인 유사도를 많이 사용한다.
""",
    "qwen2.md": """
Qwen2-7B-Instruct는 오픈 LLM 중 하나로, 한국어를 포함한 다국어 지시응답에 강하다.
RAG의 리더(생성기)로 사용하면 검색 컨텍스트를 바탕으로 간결한 답변을 생성할 수 있다.
온도(temperature)를 낮추면 사실성 유지에 도움이 된다.
"""
}
for fn, body in docs.items():
    with open(os.path.join("./kb", fn), "w", encoding="utf-8") as f:
        f.write(textwrap.dedent(body).strip() + "\n")

print("KB written to ./kb/*.md")
```

실행:

```powershell
python make_kb.py
```

---

## 2) 인덱스 빌드 (Chunk → Embedding → FAISS 저장)

* **임베딩**: `intfloat/e5-base` (간단·성능/속도 밸런스 좋음, 다국어 OK)
* **청킹**: 길이 300\~600자 권장(예시는 400자)
* **저장물**: `./index/faiss.index`(벡터), `./index/meta.npy`(문서/청크 메타)

```python
# build_index.py
import os, glob, numpy as np, faiss
from sentence_transformers import SentenceTransformer

CACHE = r"D:\WorkSpace\Hands_on_Gen_AI"
EMB_ID = "intfloat/e5-base"  # 필요 시 'bge-m3' 등으로 교체
CHUNK_SIZE = 400
OVERLAP = 50

def chunk_text(text, size=400, overlap=50):
    out = []
    start = 0
    while start < len(text):
        end = min(len(text), start + size)
        out.append(text[start:end])
        start = end - overlap
        if start < 0: start = 0
    return [c.strip() for c in out if c.strip()]

# 1) 문서 로드 & 청킹
paths = sorted(glob.glob("./kb/*.md"))
docs = []
for p in paths:
    with open(p, "r", encoding="utf-8") as f:
        t = f.read()
    chunks = chunk_text(t, CHUNK_SIZE, OVERLAP)
    for i, ch in enumerate(chunks):
        docs.append({"doc_path": p, "chunk_id": i, "text": ch})

# 2) 임베딩
embed = SentenceTransformer(EMB_ID, cache_folder=CACHE)
texts = [f"passage: {d['text']}" for d in docs]
vecs = embed.encode(texts, normalize_embeddings=True)

# 3) FAISS 인덱스(IP = 코사인과 동치 when normalized)
dim = vecs.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(vecs)

# 4) 저장
os.makedirs("./index", exist_ok=True)
faiss.write_index(index, "./index/faiss.index")
np.save("./index/meta.npy", np.array(docs, dtype=object))
print(f"Built {len(docs)} chunks, saved to ./index")
```

실행:

```powershell
python build_index.py
```

---

## 3) 질의 → 검색 → 생성(출처 포함)

* **리더(LLM)**: `Qwen/Qwen2-7B-Instruct` (오픈, 한국어 잘함)

  * GPU 없으면 더 작은 `Qwen2-1.5B-Instruct` 또는 `gemma-2-2b-it`로 바꾸세요.
* **검색**: top-k(기본 4) + 간단한 **MMR 다양성** 포함(선택)
* **프롬프트**: 사실성 강화 위해 **temperature 낮음(0.2)**, **출처 요약**해서 함께 표기

```python
# rag_query.py
import os, numpy as np, faiss, torch
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

CACHE = r"D:\WorkSpace\Hands_on_Gen_AI"
EMB_ID = "intfloat/e5-base"
LLM_ID = "Qwen/Qwen2-7B-Instruct"  # 더 작게: "Qwen/Qwen2-1.5B-Instruct"
TOP_K = 4
USE_MMR = True
MMR_LAMBDA = 0.7  # 1.0=유사도 위주, 0.0=다양성 위주

# 1) 인덱스/메타 로드
index = faiss.read_index("./index/faiss.index")
meta = np.load("./index/meta.npy", allow_pickle=True)

# 2) 임베딩 모델 로드
embed = SentenceTransformer(EMB_ID, cache_folder=CACHE)

def search(query, top_k=TOP_K, use_mmr=USE_MMR, mmr_lambda=MMR_LAMBDA):
    qv = embed.encode([f"query: {query}"], normalize_embeddings=True)
    D, I = index.search(qv, top_k*4 if use_mmr else top_k)
    cand = I[0].tolist()
    if not use_mmr:
        return cand[:top_k]

    # 간단 MMR
    selected = []
    cand_vecs = None
    # 미리 후보 벡터 모으기
    def get_vec(i):
        nonlocal cand_vecs
        if cand_vecs is None:
            # 메모리를 아끼려고, 다시 임베딩 (원본 vecs 저장 안 했으니까)
            texts = [f"passage: {meta[j]['text']}" for j in cand]
            cand_vecs = embed.encode(texts, normalize_embeddings=True)
        idx = cand.index(i)
        return cand_vecs[idx]

    q = qv[0]
    while len(selected) < top_k and cand:
        if not selected:
            best = cand[0]
            selected.append(best); cand.remove(best)
        else:
            best_i, best_score = None, -1e9
            for i in cand:
                sim_to_query = float(np.dot(get_vec(i), q))
                sim_to_selected = max(float(np.dot(get_vec(i), get_vec(s))) for s in selected) if selected else 0.0
                score = mmr_lambda * sim_to_query - (1 - mmr_lambda) * sim_to_selected
                if score > best_score:
                    best_score, best_i = score, i
            selected.append(best_i); cand.remove(best_i)
    return selected

# 3) LLM 로드
tok = AutoTokenizer.from_pretrained(LLM_ID, cache_dir=CACHE, use_fast=True)
dtype = torch.float16 if torch.cuda.is_available() else torch.float32
mdl = AutoModelForCausalLM.from_pretrained(LLM_ID, cache_dir=CACHE, torch_dtype=dtype, device_map="auto")
gen = pipeline("text-generation", model=mdl, tokenizer=tok)

def answer(query):
    hits = search(query)
    contexts = [meta[i] for i in hits]
    context_text = "\n\n---\n\n".join(
        [f"[{k+1}] {os.path.basename(c['doc_path'])} (chunk {c['chunk_id']}):\n{c['text']}" for k, c in enumerate(contexts)]
    )
    prompt = f"""당신은 신뢰할 수 있는 한국어 어시스턴트입니다.
다음 '컨텍스트'만을 근거로 간결하고 정확하게 답하세요.
가능하면 답변 끝에 [1], [2]처럼 참조 번호를 포함해 출처를 표시하세요.
모르겠으면 모른다고 답하세요.

컨텍스트:
{context_text}

질문: {query}
답변:"""
    out = gen(
        prompt, max_new_tokens=300, temperature=0.2, top_p=0.9, do_sample=True,
        repetition_penalty=1.05, pad_token_id=tok.eos_token_id
    )[0]["generated_text"]
    # prompt를 포함한 전체를 내보내니, 답만 추출(간단 파싱)
    ans = out.split("답변:")[-1].strip()
    return ans, contexts

if __name__ == "__main__":
    q = "RAG의 핵심 구성 요소는 무엇이고 왜 임베딩이 중요한가?"
    ans, ctxs = answer(q)
    print("\nQ:", q)
    print("\nA:", ans)
    print("\n[참고 출처]")
    for i, c in enumerate(ctxs, 1):
        print(f"[{i}] {os.path.basename(c['doc_path'])}#chunk{c['chunk_id']}")
```

실행:

```powershell
python rag_query.py
```

---

## 4) (선택) 재순위기(Reranker)로 품질 끌어올리기

검색 상위 8\~10개를 뽑은 뒤 **Cross-Encoder**로 상위 3\~5개만 LLM에 전달하면 답변 품질이 더 좋아집니다.

```python
# add to rag_query.py (옵션)
USE_RERANK = False
RERANK_MODEL = "cross-encoder/ms-marco-MiniLM-L-6-v2"  # pip install cross-encoder

if USE_RERANK:
    from cross_encoder import CrossEncoder
    reranker = CrossEncoder(RERANK_MODEL)

def rerank(query, indices):
    if not USE_RERANK: return indices
    pairs = [(query, meta[i]["text"]) for i in indices]
    scores = reranker.predict(pairs)
    ranked = [x for _, x in sorted(zip(scores, indices), reverse=True)]
    return ranked
```

> `search()`의 반환 직후 `indices = rerank(query, indices)` 형태로 적용.

---

## 5) 운영 팁 (짧게)

* **청킹**: 300\~600자(또는 200\~400토큰) + 10\~20% 중첩 → 회수/정확 균형
* **임베딩 프롬프트**: E5 류는 `"query: ..."`, `"passage: ..."` 프리픽스가 성능에 중요
* **하이퍼파라미터**: `TOP_K=3~5`, `temperature=0.2~0.5`, MMR로 중복 줄이기
* **출처 표기**: 문서 파일명+chunk 번호(or URL) 함께 노출
* **확장**: 문서 수↑ → **FAISS IVF/HNSW** 등 근사탐색로 전환, 다국어면 다국어 임베딩 사용
* **긴 문맥**: 긴 컨텍스트 지원 LLM(예: Qwen2 최신, Llama 계열) + FlashAttention2 활성 옵션

---

## 6) 한 줄로 요약

* 이 스크립트 세트(`make_kb.py` → `build_index.py` → `rag_query.py`)만 실행하면 **로컬 RAG**가 즉시 동작하고, 답변에 **출처**가 붙습니다.
* 다음 단계로는 **재순위기**, **긴 컨텍스트 LLM**, **파일 업로드/크롤링 파이프라인**을 차근히 붙이면 됩니다.

원하시면 이걸 **API(FastAPI) 형태**로 바꿔서 `/ask?q=...`로 호출 가능하게도 바로 만들어 드릴게요.


좋아요—**진짜 최소한**으로 “어떻게 붙이는지 감만 오게”만 적습니다. 필요해지면 키워드로 찾아 확장하면 돼요.

---

## 1) FastAPI로 `/ask?q=...` 만들기 (RAG 래핑)

**핵심 흐름**: (로드) 인덱스/임베딩/LLM → (요청) 쿼리 받기 → 검색 →(옵션: 재순위)→ 컨텍스트 붙여 LLM 생성 → JSON 반환

```python
# app.py
from fastapi import FastAPI
from pydantic import BaseModel
import faiss, numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch, os

CACHE = r"D:\WorkSpace\Hands_on_Gen_AI"
EMB_ID = "intfloat/e5-base"
LLM_ID = "Qwen/Qwen2-7B-Instruct"  # 3080이면 4bit or 더 작은 모델 권장
TOP_K = 4

app = FastAPI()

# --- 로드 (앱 시작 시 1회)
index = faiss.read_index("./index/faiss.index")
meta  = np.load("./index/meta.npy", allow_pickle=True)
embed = SentenceTransformer(EMB_ID, cache_folder=CACHE)
tok   = AutoTokenizer.from_pretrained(LLM_ID, cache_dir=CACHE, use_fast=True)
dtype = torch.float16 if torch.cuda.is_available() else torch.float32
mdl   = AutoModelForCausalLM.from_pretrained(LLM_ID, cache_dir=CACHE, torch_dtype=dtype, device_map="auto")
gen   = pipeline("text-generation", model=mdl, tokenizer=tok)

def retrieve(query, k=TOP_K):
    qv = embed.encode([f"query: {query}"], normalize_embeddings=True)
    D, I = index.search(qv, k)
    ctxs = [meta[i] for i in I[0]]
    ctx  = "\n\n---\n\n".join([f"[{j+1}] {os.path.basename(c['doc_path'])} (chunk {c['chunk_id']}):\n{c['text']}" for j,c in enumerate(ctxs)])
    return ctx, ctxs

class AskIn(BaseModel):
    q: str

@app.get("/ask")
def ask_get(q: str):
    ctx, ctxs = retrieve(q)
    prompt = f"컨텍스트만 근거로 간결히 답하세요.\n\n컨텍스트:\n{ctx}\n\n질문: {q}\n답변:"
    out = gen(prompt, max_new_tokens=300, temperature=0.2, top_p=0.9, do_sample=True)[0]["generated_text"]
    ans = out.split("답변:")[-1].strip()
    sources = [{"file": os.path.basename(c["doc_path"]), "chunk": int(c["chunk_id"])} for c in ctxs]
    return {"answer": ans, "sources": sources}
```

실행:

```powershell
uvicorn app:app --reload --port 8000
# 테스트: http://localhost:8000/ask?q=RAG가 뭔가요?
```

> 프론트에서 쓰려면 CORS 추가: `from fastapi.middleware.cors import CORSMiddleware` 로 `allow_origins=["*"]` 등 설정.

---

## 2) 재순위기(Reranker) 간단 붙이기

**아이디어**: 검색 상위 n(예: 8) → Cross-Encoder로 점수 재계산 → 상위 k(예: 3\~5)만 LLM에 전달.

```python
# 설치: pip install -U cross-encoder
from cross_encoder import CrossEncoder
RERANK_MODEL = "cross-encoder/ms-marco-MiniLM-L-6-v2"
reranker = CrossEncoder(RERANK_MODEL)

def rerank(query, indices):
    pairs = [(query, meta[i]["text"]) for i in indices]
    scores = reranker.predict(pairs)
    ranked = [x for _, x in sorted(zip(scores, indices), reverse=True)]
    return ranked

# 검색 함수에서: k를 좀 크게 뽑은 뒤 rerank → 상위 TOP_K 사용
```

---

## 3) 긴 컨텍스트 LLM으로 교체

* **방법**: LLM\_ID만 긴 컨텍스트 지원 모델로 바꾸고(예: Llama 3.1 8B/70B, Qwen2-7B-long 등), `max_new_tokens`/입력 길이 한도만 조절.
* **3080 대안**: **양자화(4bit)** 또는 **소형 롱컨텍스트 모델**.
* **프롬프트**: 긴 컨텍스트일수록 **온도 낮게(0.2\~0.5)**, **repetition\_penalty** 소폭↑.

---

## 4) 파일 업로드 파이프라인 (간단)

**흐름**: `/upload`에서 파일 수신 → 텍스트 추출 → 청킹 → 임베딩 → **기존 인덱스에 add** 또는 재빌드

```python
# pip install python-multipart pypdf
from fastapi import UploadFile, File
from pypdf import PdfReader

def extract_text_from_pdf(fp) -> str:
    r = PdfReader(fp); return "\n".join(p.extract_text() or "" for p in r.pages)

@app.post("/upload")
async def upload(file: UploadFile = File(...)):
    if file.filename.lower().endswith(".pdf"):
        text = extract_text_from_pdf(file.file)
    else:
        text = (await file.read()).decode("utf-8", errors="ignore")
    # -> chunk_text(), 임베딩 encode, faiss_index.add(vecs), meta append & 저장
    return {"ok": True, "filename": file.filename}
```

> 운영에선 비동기 작업큐(RQ/Celery)로 인덱스 업데이트를 백그라운드 처리하는 게 안정적.

---

## 5) 간단 크롤링 파이프라인

* **옵션 A(간단)**: `trafilatura`로 HTML → 텍스트 추출
* **옵션 B(뉴스/블로그)**: `newspaper3k`
* **옵션 C(사내 위키/API)**: 해당 API로 수집

```python
# pip install trafilatura
import trafilatura, requests

def fetch_text(url: str) -> str:
    html = requests.get(url, timeout=15).text
    return trafilatura.extract(html) or ""

# 사용: text = fetch_text("https://example.com/page")
# -> 파일 업로드와 동일하게 chunk → embed → index에 추가
```

---

## 6) 키워드만 더

* **FAISS 대규모**: IVF/HNSW, OPQ(양자화) 검색 속도↑ 메모리↓
* **MMR 다양성**: 중복 줄여 정보량↑
* **프롬프트 가드레일**: “컨텍스트에 없는 것은 모른다고 답해”
* **로깅/관측**: 쿼리, 히트 문서, 답변, 소요시간 기록 → 품질 개선 루프
* **배포**: `uvicorn` 단독 → `gunicorn -k uvicorn.workers.UvicornWorker` + 리버스 프록시(Nginx)

---

이 정도 틀만 잡아도 **/ask**, **업로드**, **크롤링**까지 스몰-스텝으로 붙일 수 있어요.
필요해지면 *한 부분씩* 파고들면 됩니다: “FastAPI CORS”, “Cross-Encoder rerank”, “FAISS IVF”, “trafilatura usage”, “bitsandbytes 4bit” 같은 키워드로요.
